In [ ]:
import os
import json
import numpy as np
import pandas as pd

import chart_studio.plotly as py
import plotly.graph_objects as go

In [ ]:
os.makedirs("figures", exist_ok=True)

In [ ]:
# equivalent = pd.read_excel("data/GHG_EmissionsData.xlsx", skiprows=4, index_col=0, sheet_name="CO2e")
carbon = pd.read_excel("data/GHG_EmissionsData.xlsx", skiprows=4, index_col=0, sheet_name="CO2")
methane = pd.read_excel("data/GHG_EmissionsData.xlsx", skiprows=4, index_col=0, sheet_name="CH4")
nitrous = pd.read_excel("data/GHG_EmissionsData.xlsx", skiprows=4, index_col=0, sheet_name="N2O")

In [ ]:
year = "2018"
for missing in ["NO", "NE", "NO,NA"]:
    carbon[year][carbon[year] == missing] = 0
    nitrous[year][nitrous[year] == missing] = 0
    methane[year][methane[year] == missing] = 0
    
    carbon.fillna(0.0, inplace=True)
    nitrous.fillna(0.0, inplace=True)
    methane.fillna(0.0, inplace=True)

In [ ]:
with open("data/structure.json", "r") as f:
    structure = json.load(f)

In [ ]:
carbon_scaling = 1

In [ ]:
sources = []
targets = []
value = []
labels = []


def add_items(source, target):
    if source not in labels:
        labels.append(source)
    if target not in labels:
        labels.append(target)
    sources.append(labels.index(source))
    targets.append(labels.index(target))


start = list(structure.keys())[3]
s = structure[start]

for val in s:
    add_items(start, val)
    value.append(
        abs(carbon.loc[val, year]) * carbon_scaling  +  nitrous.loc[val, year] + methane.loc[val, year]
    )
        
    # Final level
    for t in ["CO2", "N2O", "CH4"]:
        add_items(val, t)
        if t == "CO2":
            value.append(abs(carbon.loc[val, year]) * carbon_scaling)
        elif t == "N2O":
            value.append(nitrous.loc[val, year])
        elif t == "CH4":
            value.append(methane.loc[val, year])


In [ ]:
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=sources,
        target=targets,
        value=value
    ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()

In [ ]:
sources = []
targets = []
value = []
labels = []


def add_items(source, target):
    if source not in labels:
        labels.append(source)
    if target not in labels:
        labels.append(target)
    sources.append(labels.index(source))
    targets.append(labels.index(target))


start = list(structure.keys())[0]
s = structure[start]

for key, val in s.items():
    add_items(start, key)
    value.append(
        carbon.loc[key, year] * carbon_scaling +  nitrous.loc[key, year] + methane.loc[key, year]
    )

    if not val:
        for t in ["CO2", "N2O", "CH4"]:
            add_items(key, t)
            if t == "CO2":
                value.append(carbon.loc[key, year] * carbon_scaling)
            elif t == "N2O":
                value.append(nitrous.loc[key, year])
            elif t == "CH4":
                value.append(methane.loc[key, year])

    for v in val:
        value.append(
            abs(carbon.loc[v, year]) * carbon_scaling +  nitrous.loc[v, year] + methane.loc[v, year]
        )
        add_items(key, v)
        
        # Final level
        for t in ["CO2", "N2O", "CH4"]:
            add_items(v, t)
            if t == "CO2":
                value.append(abs(carbon.loc[v, year]) * carbon_scaling)
            elif t == "N2O":
                value.append(nitrous.loc[v, year])
            elif t == "CH4":
                value.append(methane.loc[v, year])


In [ ]:
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=sources,
        target=targets,
        value=value
    ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()

In [ ]:
sources = []
targets = []
value = []
labels = []


def add_items(source, target):
    if source not in labels:
        labels.append(source)
    if target not in labels:
        labels.append(target)
    sources.append(labels.index(source))
    targets.append(labels.index(target))


for start in list(structure.keys()):
    s = structure[start]

    if type(s) is dict:

        for key, val in s.items():
            add_items(start, key)
            value.append(
                carbon.loc[key, year] * carbon_scaling +  nitrous.loc[key, year] + methane.loc[key, year]
            )

            if not val:
                for t in ["CO2", "N2O", "CH4"]:
                    add_items(key, t)
                    if t == "CO2":
                        value.append(carbon.loc[key, year])
                    elif t == "N2O":
                        value.append(nitrous.loc[key, year])
                    elif t == "CH4":
                        value.append(methane.loc[key, year])

            for v in val:
                value.append(
                    carbon.loc[v, year] * carbon_scaling +  nitrous.loc[v, year] + methane.loc[v, year]
                )
                add_items(key, v)

                # Final level
                for t in ["CO2", "N2O", "CH4"]:
                    add_items(v, t)
                    if t == "CO2":
                        value.append(carbon.loc[v, year] * carbon_scaling)
                    elif t == "N2O":
                        value.append(nitrous.loc[v, year])
                    elif t == "CH4":
                        value.append(methane.loc[v, year])

    else:
        for val in s:
            add_items(start, val)
            value.append(
                abs(carbon.loc[val, year]) * carbon_scaling +  nitrous.loc[val, year] + methane.loc[val, year]
            )

            # Final level
            for t in ["CO2", "N2O", "CH4"]:
                add_items(val, t)
                if t == "CO2":
                    value.append(abs(carbon.loc[val, year]) * carbon_scaling)
                elif t == "N2O":
                    value.append(nitrous.loc[val, year])
                elif t == "CH4":
                    value.append(methane.loc[val, year])


In [ ]:
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=sources,
        target=targets,
        value=value
    ))])

fig.update_layout(title_text=f"Danish GHG Emissions in {year}", font_size=10)
fig.show()

In [ ]:
flows = pd.DataFrame(data={
    'source_id': sources,
    'target_id': targets,
    'source': [labels[s] for s in sources],
    'target': [labels[t] for t in targets],
    'value': value
})

In [ ]:
flows["node_color"] = ''
flows["link_color"] = ''

In [ ]:
flows

In [ ]:
fig = go.Figure(data=[go.Sankey(
    valueformat = ".1f",
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=sources,
        target=targets,
        value=value,
        color="lightgrey"
    ))])

fig.update_layout(title_text=f"Danish GHG Emissions in {year}", font_size=10)
fig.show(config={"responsive": True, "displayModeBar": False})

In [ ]:
fig.write_html("figures/sankey.html", config={"responsive": True, "displayModeBar": False})

In [ ]:
fig.write_image("figures/sankey.svg", width=1920, height=1080, scale=1)